In [24]:
# Import required libraries
import pandas as pd
import numpy as np

In [37]:
# Load the dataset
df = pd.read_csv("Train.csv")

In [39]:
# Load the dataset
df = pd.read_csv("Test.csv")

In [38]:
print("Train Columns:\n", df.columns.tolist())

Train Columns:
 ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales']


In [40]:
# Basic overview
print("Initial Shape:", df.shape)
df.info()
df.describe(include='all')

Initial Shape: (5681, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 488.3+ KB


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,5681,4705.000000,5681,5681.000000,5681,5681.000000,5681,5681.000000,4075,5681,5681
unique,1543,NaN,5,NaN,16,NaN,10,NaN,3,3,4
top,DRF48,NaN,Low Fat,NaN,Snack Foods,NaN,OUT027,NaN,Medium,Tier 3,Supermarket Type1
freq,8,NaN,3396,NaN,789,NaN,624,NaN,1862,2233,3717
mean,NaN,12.695633,NaN,0.065684,NaN,141.023273,NaN,1997.828903,NaN,NaN,NaN
std,NaN,4.664849,NaN,0.051252,NaN,61.809091,NaN,8.372256,NaN,NaN,NaN
min,NaN,4.555000,NaN,0.000000,NaN,31.990000,NaN,1985.000000,NaN,NaN,NaN
25%,NaN,8.645000,NaN,0.027047,NaN,94.412000,NaN,1987.000000,NaN,NaN,NaN
50%,NaN,12.500000,NaN,0.054154,NaN,141.415400,NaN,1999.000000,NaN,NaN,NaN
75%,NaN,16.700000,NaN,0.093463,NaN,186.026600,NaN,2004.000000,NaN,NaN,NaN


In [41]:
# Clean column names
df.columns = df.columns.str.strip()

In [42]:
# Strip whitespaces from categorical values
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'low fat': 'Low Fat','LF': 'Low Fat','reg': 'Regular'})

In [43]:
# Fill missing Item_Weight
item_weight_mean = df.pivot_table(values='Item_Weight', index='Item_Identifier')
missing_weight_mask = df['Item_Weight'].isnull()
df.loc[missing_weight_mask, 'Item_Weight'] = df.loc[missing_weight_mask, 'Item_Identifier'].map(item_weight_mean['Item_Weight'])
df['Item_Weight'].fillna(df['Item_Weight'].mean(), inplace=True)

In [44]:
# Fill missing Outlet_Size
outlet_size_mode = df.pivot_table(values='Outlet_Size', index='Outlet_Type', aggfunc=lambda x: x.mode().iloc[0])
missing_outlet_mask = df['Outlet_Size'].isnull()
df.loc[missing_outlet_mask, 'Outlet_Size'] = df.loc[missing_outlet_mask, 'Outlet_Type'].map(outlet_size_mode['Outlet_Size'])

In [45]:
# Fix zero visibility
zero_vis_mask = df['Item_Visibility'] == 0
item_vis_mean = df.pivot_table(values='Item_Visibility', index='Item_Identifier')
df.loc[zero_vis_mask, 'Item_Visibility'] = df.loc[zero_vis_mask, 'Item_Identifier'].map(item_vis_mean['Item_Visibility'])

In [46]:
# Normalize 'Item_Fat_Content'
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'low fat': 'Low Fat','LF': 'Low Fat','reg': 'Regular'})
print("Unique values in Item_Fat_Content after cleaning:", df['Item_Fat_Content'].unique())

Unique values in Item_Fat_Content after cleaning: ['Low Fat' 'Regular']


In [47]:
# Final cleaned dataset for EDA
eda_df = df.copy()

In [48]:
# Final check
print("\n Missing Values After Cleaning:")
print(df.isnull().sum())

print("\n Unique Fat Content Values:")
print(df['Item_Fat_Content'].unique())


 Missing Values After Cleaning:
Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

 Unique Fat Content Values:
['Low Fat' 'Regular']


In [49]:
# 9. Preview cleaned data
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,Regular,0.038428,Dairy,87.3198,OUT017,2007,Small,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Small,Tier 2,Supermarket Type1
4,FDY38,13.600,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3
